In [ ]:
from sympy.combinatorics import *
from itertools import combinations
from sympy.interactive import init_printing

from tqdm.notebook import tqdm

from math import factorial
import numpy as np

import pickle

init_printing(perm_cyclic=True, pretty_print=False) #prints permutations in compact notation
perm_type = permutations.Permutation

# Reproduce (correct) coefficients from project catalog

In [ ]:
def gen_ctype(l: dict,n: int):
    '''
    Input:
        l - cycle decompostion as a dictionary including 1 cycles. e.g. {1:2, 2:1} is a transposition in S_4
        n - specifies order of symmetric group
    Output:
        Generates all permutations of specified cycletype as a list
    '''
    
    has_to_be_partition = 0
    for key, value in l.items():
        has_to_be_partition = has_to_be_partition + key*value
    if has_to_be_partition < n:
        print("Invalid cycle type format.")
    else:
        list_with_cycle_type = []
        
        #run through all permutations and append them if they have the correct cycletype
        for i in list(symmetric(n)): 
            if i.cycle_structure == l:
                list_with_cycle_type.append(i)            
        return list_with_cycle_type
    
Permutation(3)(0,1).cycle_structure, gen_ctype({1:2,2:1},4)

In [ ]:
def norm(p, cycle_dict = None):
    '''
    Input: permutation p, optionally the cycle structure of p.
    Output: The norm of p
    '''
    #If the cycle type of p is already computed we may reuse the computation.
    if cycle_dict == None:
        cycle_dict = p.cycle_structure
    sum_of_lengths = 0
    num_cycles = 0
    for key, value in cycle_dict.items():
        sum_of_lengths += key*value
        num_cycles += value
    return sum_of_lengths-num_cycles

In [ ]:
def norm_check(p,q, norm_sum = None):
    if norm_sum == None:
        norm_sum = norm(p) + norm(q)
    return norm(p*q)==norm_sum

In [ ]:
def norm_check2(p,q, norm_sum = None):
    if norm_sum == None:
        norm_sum = norm(p) + norm(q)
    p_trans = set(p.transpositions())
    q_trans = set(q.transpositions())
    
    if p_trans.intersection(q_trans) == set([]):
        return norm(p*q)==norm_sum
    return False

In [ ]:
def myprod(p_list, q_list, norm_sum = None):
    if norm_sum:
        return [s*t for s in p_list for t in q_list if norm_check2(s,t,norm_sum)]
    return [s*t for s in p_list for t in q_list if norm_check2(s,t)]

In [ ]:
# Create element g2 as a list of permutations (sympy permutations)
g2 = gen_ctype({1:2, 2:1},4)
g2

In [8]:
# Generate powers of g2
g2_square = myprod(g2,g2,2)
g2_cube = myprod(g2,g2_square,3)
g2_quad = myprod(g2,g2_cube,4)

In [9]:
def memoize(f):
    memo = {}
    def helper(x):
        if x not in memo:            
            memo[x] = f(x)
        return memo[x]
    return helper

In [10]:
def ctype(p: perm_type):
    '''
    Input:
        Sympy permutation
    Output:
        Gives cycletype of permutation as a tuple with i'th coordinate being multiplicity of i-cycles.
    '''
    d = p.cycle_structure
    max_val = max(d.keys())
    L = []
    for i in range(1,max_val+1):
        if i in list(d.keys()):
            L.append(d[i])
        else:
            L.append(0)
    return tuple(L)

ctype(Permutation(3)(1,3)) # has 2 1-cycles (0 and 2) and one 2-cycle

(2, 1)

In [11]:
def decompose(plist: list, progressbar=True):
    '''
    Input: List of permutations
    Output: Decomposes list by occurences of cycletype
    '''
    cycle_count = {}
    for p in tqdm(plist, disable = progressbar):
        p_type = ctype(p)
        if p_type in list(cycle_count.keys()):
            cycle_count[p_type] += 1
        else:
            cycle_count[p_type] = 1
    return cycle_count

print("g2 squared decomp", decompose(g2_square),
      "\n" "g2 cubed decomp", decompose(g2_cube))

g2 squared decomp {(1, 0, 1): 24, (0, 2): 6} 
g2 cubed decomp {(0, 0, 0, 1): 96}


In [12]:
def num_ctype(ctuple):
    '''
    Input: cycle type as a tuple
    Output: number of permutations with given cycle type using 
    '''
    n = sum((i+1)*ctuple[i] for i in range(len(ctuple)))
    numerator = factorial(n)
    denominator = 1
    for i in range(len(ctuple)):
        denominator *= ((i+1)**ctuple[i]) * factorial(ctuple[i])
    return numerator // denominator

num_ctype((2,1))

6

In [13]:
def to_string(d):
    if type(d) == dict:
        s = ""
        for key, value in d.items():
            s += str(value) + "*" + "g_"+ to_string(key) + "  +  "
        return s[0:-3]
    s = ""
    for i,k in enumerate(d):
        if i!=0 and k!= 0:
            s+= str(i+1) + "^" + str(k) + ","
    return s[0:-1]

In [14]:
def decompose2(plist, string_output = False):
    '''
    Input: list of permutations
    Output: decomposition according to A(d)
    '''
    d = decompose(plist)
    for i in d.keys():
        d[i] //= num_ctype(i)
        
    if string_output:
        return to_string(d)
    return d

In [15]:
print("g2 square", decompose2(g2_square), 
      "\n" "g2 cube", decompose2(g2_cube), 
      "\n" "g2 quad", decompose2(g2_quad))

g2 square {(1, 0, 1): 3, (0, 2): 2} 
g2 cube {(0, 0, 0, 1): 16} 
g2 quad {}


## Output coefficient matrix

In [ ]:
d = 6
g2 = gen_ctype({1:(d-2), 2:1}, d)

g2_powers = [myprod(g2,g2,2)]

for i in tqdm(range(3,3+3)):
    g2_powers.append(myprod(g2,g2_powers[-1],i))

In [ ]:
g2_powers_decomp = []
for i in tqdm(g2_powers):
    g2_powers_decomp.append(decompose2(i))
    print(g2_powers_decomp[-1])

In [ ]:
all_keys = set([])
for decomp in g2_powers_decomp:
    all_keys = all_keys.union(set(decomp.keys()))
all_keys = all_keys - {(d-2, 1), (d,)}
all_keys = list(all_keys)
all_keys.sort()
all_keys

In [ ]:
(n,m) = (len(g2_powers_decomp),len(all_keys))
A = np.zeros((n,m)).astype(int)

for i in range(n):
    for j in range(m):
        try:
            A[i,j] = (g2_powers_decomp[i])[all_keys[j]]
        except:
            pass

A

In [ ]:
np.linalg.matrix_rank(A)

# Relations

In [16]:
d = 6
generators = []

for i in tqdm(range(2, d//2+2)):
    generators.append(
        gen_ctype({1:(d-i), i:1}, d)
    )

In [17]:
for i,g1 in enumerate(generators):
    for j,g2 in enumerate(generators[i:]):
        print(i+2,j+2,decompose2(myprod(g1,g2,i+j+2)))

2 2 {(3, 0, 1): 3, (2, 2): 2}
2 3 {(2, 0, 0, 1): 4, (1, 1, 1): 1}
2 4 {(1, 0, 0, 0, 1): 5, (0, 1, 0, 1): 1}
3 2 {}
3 3 {}
4 2 {(1, 0, 0, 0, 1): 9, (0, 0, 2): 18, (0, 1, 0, 1): 10}


In [18]:
decompose2(myprod(generators[0], gen_ctype({1:d-4, 2:2}, d)))

{(0, 3): 3, (1, 1, 1): 3, (2, 0, 0, 1): 2}

In [19]:
def gen_all(d):
    ''' Input: non-negative interger
        Output: Dictionary of elements in the symmetric group with key being cycle type and value all permutations in S(d) with
        that cycle type.
    '''
    answer = {}
    
    for i in tqdm(symmetric(d), total = factorial(d)):
        C = ctype(i)
        if C in answer.keys():
            (answer[C]).append(i)
        else:
            answer[C] = [i]
    answer.pop((d,),None)
    return answer

def norm2(p_tuple):
    ans = 0
    for i,p in enumerate(p_tuple):
        ans += i*p
    return ans

In [ ]:
d=9
basis = gen_all(d)

In [ ]:
g2_mult_table = []

for k,v in tqdm(basis.items()):
    s ="g_2 * g_" + to_string(k) + " = "
    s += decompose2(myprod(basis[(d-2,1)],v, norm2((d-2,1)) + norm2(k)), string_output = True)
    g2_mult_table.append(s)
    print(s)

In [ ]:
#with open('g2table.pkl', 'wb') as f:
#    pickle.dump(g2_mult_table, f)

In [ ]:
for k1,v1 in basis.items():
    for k2,v2 in basis.items():
        if k1 >= k2:
            #print(k1,k2, "\n", decompose2(myprod(v1,v2, norm2(k1) + norm2(k2)), string_output = True))
            pass

In [ ]:
products = []
for i in range(1,d//2+1):
    p = basis[(d-2*i, i)]
    for j in range(i,d//2+1):
        q = basis[(d-2*j, j)]
        dic = decompose2(myprod(p,q, i + j))
        products.append([i,j, dic])
        print(i,j, to_string(dic))

In [ ]:
#with open('prod_obs.pkl', 'wb') as f:
#    pickle.dump(products, f)

In [ ]:
with open('g2table.pkl', 'rb') as f:
    products = pickle.load(f)
for line in products:
    print(line[0:-3])

In [ ]:
for line in products:
    print(line[0],line[1], list(line[2].values()))

In [ ]:
g23 = basis[(4,3)]
g22 = basis[(6,2)]

In [ ]:
decompose2(myprod(g22,g23,5))

# Relations 2.0

In [20]:
d=6
basis = gen_all(d)

In [21]:
x = basis[(d-2,1)]
y = basis[(d-4,2)]
z = basis[(d-6,3)]

In [22]:
f = lambda u, v: myprod(u,v)
g = lambda u: print(decompose2(u, string_output = True))


print("w = 2")
x2 = f(x,x)
g(x2)

print("w = 3")
x3 = f(x2,x)
g(x3)

xy = f(x,y)
g(xy)

print("w = 4")
x4 = f(x3,x)
g(x4)

x2y = f(x,xy)
g(x2y)

y2 = f(y,y)
g(y2)

xz = f(x,z)
g(xz)

print("w = 5")
g(f(x4,x))
g(f(x3,y))
g(f(x,y2))
g(f(x,xz))
g(f(y,z))


w = 2
3*g_3^1  +  2*g_2^2  
w = 3
16*g_4^1  +  9*g_2^1,3^1  +  6*g_2^3  
2*g_4^1  +  3*g_2^1,3^1  +  3*g_2^3  
w = 4
125*g_5^1  +  64*g_2^1,4^1  +  54*g_3^2  
25*g_5^1  +  20*g_2^1,4^1  +  18*g_3^2  
5*g_5^1  +  4*g_2^1,4^1  +  9*g_3^2  
2*g_2^1,4^1  
w = 5
1296*g_6^1  
324*g_6^1  
81*g_6^1  
12*g_6^1  
3*g_6^1  


# Trashcan

In [ ]:
def test_ctype(p: perm_type, D):
    '''
    Input:
        p
    '''
    d = p.cycle_structure
    d.pop(1, None)
    if d.keys() == D.keys():
        for key in D.keys():
            if d[key] != D[key]:
                return False
        return True
    return False